# Análise de Dados - Olist

Este jupyter notebook apresenta uma análise geral dos dados da plataforma de vendas Olist, organizando e visualizando informações sobre clientes, produtos e vendas.

### 1. Importação das Bibliotecas e Leitura dos Dados

Carregamos os arquivos CSV para os respectivos DataFrames.

In [38]:
import pandas as pd
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('expand_frame_repr', False)

caminho_arquivos = {
    "clientes": "dados/olist_customers_dataset.csv",
    "geolocalizacao": "dados/olist_geolocation_dataset.csv",
    "itens_pedidos": "dados/olist_order_items_dataset.csv",
    "pagamentos_pedidos": "dados/olist_order_payments_dataset.csv",
    "avaliacoes_pedidos": "dados/olist_order_reviews_dataset.csv",
    "pedidos": "dados/olist_orders_dataset.csv",
    "produtos": "dados/olist_products_dataset.csv",
    "vendedores": "dados/olist_sellers_dataset.csv",
    "traducao_categorias_produtos": "dados/product_category_name_translation.csv",
}

dataframes = {nome: pd.read_csv(caminho) for nome, caminho in caminho_arquivos.items()}

clientes_df = dataframes["clientes"]
geolocalizacao_df = dataframes["geolocalizacao"]
itens_pedidos_df = dataframes["itens_pedidos"]
pagamentos_pedidos_df = dataframes["pagamentos_pedidos"]
avaliacoes_pedidos_df = dataframes["avaliacoes_pedidos"]
pedidos_df = dataframes["pedidos"]
produtos_df = dataframes["produtos"]
vendedores_df = dataframes["vendedores"]
traducao_categorias_produtos_df = dataframes["traducao_categorias_produtos"]

### 2. Pré-visualização dos Dados

Exibimos uma amostra de cada DataFrame para entender sua estrutura e conteúdo.

In [39]:
clientes_df.sample(2)

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
68452,3c2c0a8240bf76896722b75881b676d1,54ead41b207a88ee792c5e5443444d3d,89036,blumenau,SC
25604,0e077e2c9e518b1443382b33e00ba95c,212b42cfcacfe76f9c92ee59abf8ab54,2051,sao paulo,SP


In [40]:
geolocalizacao_df.sample(2)

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
691205,45655,-14.818091,-39.027095,ilheus,BA
110858,4828,-23.727189,-46.700565,são paulo,SP


In [41]:
itens_pedidos_df.sample(2)

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
38960,588c0e3b460d83b9cd8a270edc0f4da0,3,35557c68a22ecebcf066e25ca2ddc144,f8db351d8c4c4c22c6835c19a46f01b0,2017-11-14 13:55:25,16.9,34.15
56585,80b74dde5b336c1f75c610521dedec08,1,f7f59e6186e10983a061ac7bdb3494d6,729f06993dac8e860d4f02d7088ca48a,2018-02-01 21:58:02,34.9,13.64


In [42]:
pagamentos_pedidos_df.sample(2)

,order_id,payment_sequential,payment_type,payment_installments,payment_value
11631,47ff8d381edb0deb837f57d7805f0a56,1,debit_card,1,96.80
97478,dec806358e32446003f5fea8111b10e8,1,credit_card,9,336.66


In [43]:
avaliacoes_pedidos_df.sample(2)

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
79797,04a15fdeccc0572ca4f52b16cd50408a,0011d82c4b53e22e84023405fb467e57,4,NaN,NaN,2018-01-27 00:00:00,2018-01-28 13:18:49
76617,81961c5c0ea7faa926c8c9ec5ee3d0ae,ea39d327906c9e8540a9b0f5004dcf46,4,NaN,NaN,2017-08-19 00:00:00,2017-08-19 23:31:03


In [44]:
pedidos_df.sample(2)

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
63843,99fe8093159dc446655c68b320222c8e,0c9712bf6ecf766c8e85d92f7991eb54,delivered,2017-04-28 17:12:38,2017-04-28 17:22:55,2017-05-02 16:56:08,2017-05-08 10:52:01,2017-06-02 00:00:00
79304,e145c6c37dcc3fca500f3002c65e5e6c,54c8e2af42de8737d1e0fbbef5e7416c,delivered,2018-02-17 12:14:11,2018-02-17 12:26:51,2018-02-19 20:38:30,2018-03-01 00:27:19,2018-03-19 00:00:00


In [45]:
produtos_df.sample(2)

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
20967,a58b5955d37aea2de1ff80d5cba4f8ba,moveis_decoracao,41.0,900.0,1.0,500.0,27.0,35.0,22.0
20368,41b24e4f49504cc023666ed7cde3cf16,esporte_lazer,52.0,628.0,7.0,1800.0,25.0,30.0,40.0


In [46]:
vendedores_df.sample(5)

,seller_id,seller_zip_code_prefix,seller_city,seller_state
2218,bbaff50f3b708fda865918715276cd87,12940,atibaia,SP
1668,4cc4fd4fdd406a85bbdc1f824b731bd7,85906,toledo,PR
276,a3e9a2c700480d9bb01fba070ba80a0e,14940,ibitinga,SP
1039,26bad72add1f86e08a776ce4ffdaf1ca,14470,pedregulho,SP
1950,0eaa7e33d73809f1f925f7d07c506cbd,9111,santo andre,SP


In [47]:
traducao_categorias_produtos_df.sample(2)

,product_category_name,product_category_name_english
7,utilidades_domesticas,housewares
41,instrumentos_musicais,musical_instruments


In [48]:
produtos_df.describe()

,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
count,32341.000000,32341.000000,32341.000000,32949.000000,32949.000000,32949.000000,32949.000000
mean,48.476949,771.495285,2.188986,2276.472488,30.815078,16.937661,23.196728
std,10.245741,635.115225,1.736766,4282.038731,16.914458,13.637554,12.079047
min,5.000000,4.000000,1.000000,0.000000,7.000000,2.000000,6.000000
25%,42.000000,339.000000,1.000000,300.000000,18.000000,8.000000,15.000000
50%,51.000000,595.000000,1.000000,700.000000,25.000000,13.000000,20.000000
75%,57.000000,972.000000,3.000000,1900.000000,38.000000,21.000000,30.000000
max,76.000000,3992.000000,20.000000,40425.000000,105.000000,105.000000,118.000000


### 3. Identificação de Valores Ausentes

Identificamos valores ausentes em cada coluna dos DataFrames e organizamos os resultados em uma tabela consolidada para melhor visualização.

In [49]:
resumo = []

for nome, df in dataframes.items():
    valores_ausentes = df.isnull().sum() 
    df_resumo = pd.DataFrame({
        "DataFrame": [nome] * len(valores_ausentes),
        "Coluna": valores_ausentes.index,
        "Valores ausentes": valores_ausentes.values
    })
    resumo.append(df_resumo)
    
df_resumo_final = pd.concat(resumo, ignore_index=True)
df_resumo_final


,DataFrame,Coluna,Valores ausentes
0,clientes,customer_id,0
1,clientes,customer_unique_id,0
2,clientes,customer_zip_code_prefix,0
3,clientes,customer_city,0
4,clientes,customer_state,0
5,geolocalizacao,geolocation_zip_code_prefix,0
6,geolocalizacao,geolocation_lat,0
7,geolocalizacao,geolocation_lng,0
8,geolocalizacao,geolocation_city,0
9,geolocalizacao,geolocation_state,0


### 4. Tratamento de Dados

#### 4.1 Substituição de Valores Ausentes

In [50]:
avaliacoes_pedidos_df['review_comment_message'] = avaliacoes_pedidos_df['review_comment_message'].fillna('Sem Comentário')
avaliacoes_pedidos_df['review_comment_title'] = avaliacoes_pedidos_df['review_comment_title'].fillna('Sem Título')

In [51]:
# preenche valores ausentes com a média nas colunas numéricas do DataFrame 'produtos_df'
colunas_numericas = ['product_name_lenght', 'product_description_lenght', 'product_photos_qty', 'product_weight_g', 'product_length_cm', 'product_height_cm', 'product_width_cm']

produtos_df[colunas_numericas] = produtos_df[colunas_numericas].apply(lambda col: col.fillna(col.mean()))

### 5. Análise Exploratória de Dados (EDA)

Exploramos os dados para obter _insights_ sobre tempos de entrega, vendas por categoria e vendas por localização geográfica.

#### 5.1 Tempo Médio de Entrega

Calculamos o tempo médio de entrega com base nas datas de compra e entrega ao cliente:

In [52]:
pedidos_df['order_delivered_customer_date'] = pd.to_datetime(pedidos_df['order_delivered_customer_date'])
pedidos_df['order_purchase_timestamp'] = pd.to_datetime(pedidos_df['order_purchase_timestamp'])
pedidos_df['delivery_time'] = (pedidos_df['order_delivered_customer_date'] - pedidos_df['order_purchase_timestamp']).dt.days

tempo_medio_entrega = pedidos_df['delivery_time'].mean()
tempo_medio_entrega = round(tempo_medio_entrega, 2)
print(f"Tempo médio de entrega: {tempo_medio_entrega} dias")

Tempo médio de entrega: 12.09 dias


#### 5.2 Vendas por Categoria de Produto

Analisamos a frequência de vendas por categoria de produto, integrando os dados de itens de pedidos e produtos.

In [53]:
itens_produtos = pd.merge(itens_pedidos_df, produtos_df, on='product_id', how='inner')

vendas_por_categoria = itens_produtos['product_category_name'].value_counts()

vendas_por_categoria_df = pd.DataFrame(vendas_por_categoria).reset_index()
vendas_por_categoria_df.columns = ['Categoria de Produto', 'Frequência de Vendas']

vendas_por_categoria_df


,Categoria de Produto,Frequência de Vendas
0,cama_mesa_banho,11115
1,beleza_saude,9670
2,esporte_lazer,8641
3,moveis_decoracao,8334
4,informatica_acessorios,7827
5,utilidades_domesticas,6964
6,relogios_presentes,5991
7,telefonia,4545
8,ferramentas_jardim,4347
9,automotivo,4235


#### 5.3 Vendas por Localização Geográfica

Exploramos a distribuição das vendas com base nas informações de localização de clientes e geolocalização.

##### Por estado:

In [54]:
vendas_geolocalizacao = pd.merge(clientes_df, geolocalizacao_df, 
                                  left_on='customer_zip_code_prefix', 
                                  right_on='geolocation_zip_code_prefix', 
                                  how='inner')

vendas_por_estado = vendas_geolocalizacao['geolocation_state'].value_counts()

vendas_por_estado_df = pd.DataFrame(vendas_por_estado).reset_index()
vendas_por_estado_df.columns = ['Estado', 'Frequência de Vendas']

vendas_por_estado_df


,Estado,Frequência de Vendas
0,SP,5620430
1,RJ,3015690
2,MG,2878728
3,RS,805370
4,PR,626021
5,SC,538638
6,BA,365875
7,ES,316654
8,GO,133146
9,MT,122395


#### Por cidade:

In [55]:
vendas_por_cidade = vendas_geolocalizacao['geolocation_city'].value_counts()

vendas_por_cidade_df = pd.DataFrame(vendas_por_cidade).reset_index()
vendas_por_cidade_df.columns = ['Cidade', 'Frequência de Vendas']

vendas_por_cidade_df.head(50)


,Cidade,Frequência de Vendas
0,rio de janeiro,1913913
1,sao paulo,1164470
2,belo horizonte,737556
3,niteroi,393175
4,curitiba,255731
5,santos,238952
6,porto alegre,228803
7,são paulo,207882
8,campinas,170318
9,uberlandia,160498
